## Строим структуру заголовков и текста в соответствии со входным файлом.

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

In [ ]:
%run extract_formulas_urls.ipynb

#### Здесь генерируется xml-файл, но в Jupyter Notebook это почему-то не работает.

Можно заменить командой в командной строке:
<code>pdf2txt.py -o *graduationWorkXMLFile* *graduationWorkPDFFile*</code>. Генерировать нужно в папку с исходным файлом.

In [ ]:
import subprocess

subprocess.check_call(["pdf2txt.py", "-o", graduationWorkXMLFile, graduationWorkPDFFile])

### Извлекаем все заголовки из текста и строим структуру текста в соотвествии с иерархией заголовков.

In [ ]:
import xml.etree.ElementTree as etree
from collections import OrderedDict
from pprint import pprint
    
def extractHeaders(graduationWorkXMLFile):
    headers = OrderedDict()
    headerPatterns = []
    headersByOrder = []

    with open(graduationWorkXMLFile, 'r') as graduationWorkXML:
        graduationWorkXMLTree = etree.parse(graduationWorkXML) 

        extractHeaders = False # обрабатываем ли мы сейчас оглавление?
        firstHeader = ''
        textItem = ''
        headerNumeration = ''
        lastInsertPlace = ""
        lastInsertNumeration = ""
        pageNumber = -1

        # вытаскиваем заголовки из оглавления
        for page in graduationWorkXMLTree.getroot():
            pageNumber += 1
            for textbox in page:
                for textline in textbox:
                    # собираем заголовок по буквам
                    for text in textline:
                        try:
                            textItem += text.text
                        except:
                            pass
#                     print(repr(textItem))

                    # проверяем, рассматриваем ли мы сейчас оглавление
                    if textItem.lower().strip() == 'оглавление' or \
                       textItem.lower().strip() == 'содержание':
                        extractHeaders = True
                        textItem = ''
                        continue

                    # пропускаем текстовый элемент, если он не относится к оглавлению или пуст
                    if not extractHeaders or textItem == '' or textItem.isspace():
                        textItem = ''
                        continue

                    # проверяем, содержит ли сборный заголовок все необходимое
                    contentItem = textItem.replace('-\n', '').replace('\n', ' ')
                    if contentItem[-1] == '-': # на случай наличия переносов слов в оглавлении
                        contentItem = contentItem[:-1]           

                    # заголовок без нумерации
                    contentItemWithoutNumeration = contentItem
                    if not set(contentItem.split(' ', 1)[0]).issubset(numbersAndPoint):
                        # считаем этот случай продолжением предыдущего заголовка, если такая строка начинается с маленькoй буквы
                        if contentItem[0] in russianLetters or contentItem[0] in englishLetters:
                            header = contentItemWithoutNumeration.replace('..', ' ')
                            header = header.replace(' .', ' ')
    #                         print(lastInsertPlace)
                            while (header != header.replace('  ', ' ')):
                                header = header.replace('  ', ' ')
                            if set(header.rsplit(' ', 1)[-1]).issubset(numbers):
                                header = header.rsplit(' ', 1)[0]
                            exec(lastInsertPlace + "+= '" + header + "'")
                            textItem = ''
                            continue
                        # или считаем, что номер был на предыдущей строке
                        if (contentItem[0] in russianUpperLetters or contentItem[0] in englishUpperLetters) and \
                           headerNumeration != '' and lastInsertNumeration != '':
                            # разрешаем так делать только если похоже, что нумерация следует за последним добавленным заголовком
                            headerNumerationParts = headerNumeration.split('.')
                            lastInsertNumerationParts = lastInsertNumeration.split('.')
                            if set(headerNumeration).issubset(numbersAndPoint):
                                if len(headerNumerationParts) == len(lastInsertNumerationParts) + 1:
                                    for index in range(len(lastInsertNumerationParts)):
                                        if headerNumerationParts[index] != lastInsertNumerationParts[index]:
                                            headerNumeration = ''
                                            break
                                    else:
                                        contentItem = headerNumeration + ' ' + contentItem
                                if len(headerNumerationParts) + 1 == len(lastInsertNumerationParts):
                                    for index in range(len(headerNumerationParts) - 1):
                                        if headerNumerationParts[index] != lastInsertNumerationParts[index]:
                                            headerNumeration = ''
                                            break
                                    else:
                                        if int(headerNumerationParts[len(lastInsertNumerationParts) - 1]) != \
                                           int(lastInsertNumerationParts[len(lastInsertNumerationParts) - 1]) + 1:
                                            headerNumeration = ''
                                        else:
                                            contentItem = headerNumeration + ' ' + contentItem
                                if len(headerNumerationParts) == len(lastInsertNumerationParts):
                                    for index in range(len(lastInsertNumerationParts) - 1):
                                        if headerNumerationParts[index] != lastInsertNumerationParts[index]:
                                            headerNumeration = ''
                                            break
                                    else:
                                        if int(headerNumerationParts[len(lastInsertNumerationParts) - 1]) != \
                                           int(lastInsertNumerationParts[len(lastInsertNumerationParts) - 1]) + 1:
                                            headerNumeration = ''
                                        else:
                                            contentItem = headerNumeration + ' ' + contentItem

#                     print(repr(contentItem))

                    # заголовок c нумерацией
                    if set(contentItem.split(' ', 1)[0]).issubset(numbersAndPoint):
                        if not (len(contentItem.split(' ', 1)) > 1) or \
                           contentItem.split(' ', 1)[1].isspace() or contentItem.split(' ', 1)[1] == '':
                            headerNumeration = contentItem.split(' ', 1)[0]
                            textItem = ''
                            # в текущей версии элемент оглавления содержит только нумерацию, нужно рассмотреть следующую строку
                            continue
                        headerNumeration = contentItem.split(' ', 1)[0]
                        contentItemWithoutNumeration = contentItem.split(' ', 1)[1]     

                    # это не надежный момент, презентация может быть сверстана очень криво
                    # предположим, что в конце элемента оглавления должен быть номер страницы / несколько пробелов / несколько точек
    #                 if not (set(contentItemWithoutNumeration.rsplit(' ', 1)[-1]).issubset(numbers) or \
    #                      contentItemWithoutNumeration[::-1].startswith('  ') or \
    #                      contentItemWithoutNumeration[::-1].startswith('..')):
    #                     # в текущей версии элемента оглавления не содержит признак конца
    #                     continue

                    # удаляем из элемента контента все ненужное, оставляем только сам заголовок
                    header = contentItemWithoutNumeration.replace('..', ' ')
                    header = header.replace(' .', ' ')
                    while (header != header.replace('  ', ' ')):
                        header = header.replace('  ', ' ')
                    header = header.strip()
                    if set(header.rsplit(' ', 1)[-1]).issubset(numbers):
                        header = header.rsplit(' ', 1)[0]

                    # проверяем, не превратился ли заголовок в номер (особенности верстки)
                    if set(header).issubset(numbers):
                        headerNumeration = ''
                        textItem = ''
                        continue

                    # проверяем, закончили ли мы просматривать оглавление
                    if extractHeaders and (firstHeader == header.lower() or \
                                           header.lower() == 'аннотация'):
                        return(headers, headerPatterns, headersByOrder, pageNumber)

                    # добавляем место для очередного заголовка в структуре заголовков
                    newHeaderPlaceLink = headers
                    for headerNumerationPart in headerNumeration.split('.'):
                        if headerNumerationPart not in newHeaderPlaceLink.keys():
                            newHeaderPlaceLink[headerNumerationPart] = [{'header': '', 'text': ''}, OrderedDict()]
                            newHeaderPlaceLink = newHeaderPlaceLink[headerNumerationPart][1]
                        else:
                            newHeaderPlaceLink = newHeaderPlaceLink[headerNumerationPart][1]

                    # добавляем новые заголовки через exec, потому что не знаем заранее уровень вложенности
                    addHeaderToHeaders = 'headers'
                    for headerNumerationPart in headerNumeration.split('.'): 
                        addHeaderToHeaders += "['" + headerNumerationPart + "'][1]"
                    addHeaderToHeaders = addHeaderToHeaders[:-2] 
                    lastInsertPlace = addHeaderToHeaders + "0]"
                    if eval(lastInsertPlace + "['header']") == '':
                        addHeaderToHeaders += "0]['header'] = '" + header + "'"
                    elif isinstance(eval(lastInsertPlace + "['header']"), list):
                        addHeaderToHeaders += "0]['header'] = [" + lastInsertPlace + "['header'], '" + header + "']"
                    else:
                        addHeaderToHeaders += "0]['header'] = " + lastInsertPlace + "['header'].append(" + header + ")"
                    exec(addHeaderToHeaders)
                    lastInsertNumeration = headerNumeration
                    headerPatterns.append({
                        'headerPattern': (lastInsertNumeration + ' ' + header).strip(),
                        'text': ''
                    })
                    headersByOrder.append((lastInsertNumeration + ' ' + header).strip())
#                     print(lastInsertNumeration)

                    # сохраняем первый заголовок, чтобы идентифицировать потом конец оглавления
                    if extractHeaders and firstHeader == '' and header != '':
                        firstHeader = header.lower()
                        headerNumeration = ''
                        textItem = ''  
                        continue

                    headerNumeration = ''
                    textItem = ''

In [ ]:
from pprint import pprint

headersHierarchy, headerPatterns, headersByOrder, rememberContentPage = extractHeaders(graduationWorkXMLFile)

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersHierarchy)
    pprint(headerPatterns)
    pprint(headersByOrder)

### Извлекаем куски текста, соответствующие выделенным заголовкам.

In [ ]:
def formatTextPart(textPart, text):
    # замеченные неточности в парсинге символов
    formattedText = textPart.replace('(cid:22)', '-').replace('(cid:88)\n', '').replace('(cid:190)', '«').replace('¿', '»')
    formattedText = formattedText.replace('\uf0b7', '•')
    # это попытка обрезать цифры, соответствующие сноскам
    notes = re.findall(r'[а-яА-ЯёЁ]+\d', formattedText)
    for note in notes:
        formattedText = formattedText.replace(note, re.sub(r'\d', '', note))

    # это попытка убрать сломанные формулы
    for formula in formulas:
        formattedText = formattedText.replace(formula, '*formula placeholder*.')

    # это попытка сохранить форматирование для элементов списков
    formattedTextTemp = ''
    for formattedTextPart in list(filter(None, formattedText.split('\n'))):
        formattedTextPart = formattedTextPart.strip()
#         print(formattedTextPart, formattedTextPart != '', )
        if formattedTextPart != '' and \
           (len(formattedTextPart.split(' ', 1)) > 1) and \
           (len(formattedTextPart.split(' ', 1)[0]) > 0) and \
           (formattedTextPart.split(' ', 1)[1].strip()) and \
           (formattedTextPart.split(' ', 1)[0][0] in numbers or \
            formattedTextPart.split(' ', 1)[0][0] in listMarkers or \
            formattedTextPart.split(' ', 1)[-1][-1] == ':'):
            if formattedTextPart.split(' ', 1)[-1][-1] == ':':
                formattedTextTemp += ' ' + formattedTextPart + '\n\n'
                continue
            formattedTextTemp += '\n\n' + formattedTextPart 
        else:
            formattedTextTemp += ' ' + formattedTextPart

    formattedText = formattedTextTemp
        
    return formattedText

def splitTextByHeaderPatterns(headerPatternsAndText, pageObjectsByY1List, contentPageNumber, formulas):    
    
    for headerPatternIndex in range(0, len(headerPatterns)):
        extractText = False
        textObjectsAccumulation = []
        for pageNumber in range(contentPageNumber, len(pageObjectsByY1List)):
            pageObjectsKeys = [key for key in pageObjectsByY1List[pageNumber].keys()]
            textUnderLine = False
            for y1, pageObjects in pageObjectsByY1List[pageNumber].items():
                for pageObjectIndex in range(0, len(pageObjects)):
                    pageObject = pageObjects[pageObjectIndex]
                    
                    # обычно сноски идут под линией перед номером страницы
#                     if isinstance(pageObject['object'], LTLine):
#                         textUnderLine = not textUnderLine
#                         if not textUnderLine and extractText:
#                             for textPiece in textObjectsAccumulation[::-1]:
#                                 # это означает, что тут не было сноски, а текст не был обработан ранее обработан
#                                 if textPiece['label'] == 'possible note':
#                                     textPiece['label'] = ''
#                                     headerPatterns[headerPatternIndex]['text'] += formatTextPart(
#                                         textPiece['object'].get_text(), 
#                                         headerPatterns[headerPatternIndex]['text']
#                                     )
                                    
                    if isinstance(pageObject['object'], LTTextBox) and \
                       (pageObject['label'] != 'table caption' and \
                        pageObject['label'] != 'image caption' and \
                        pageObject['label'] != 'table part'):
                        objectTextPart = pageObject['object'].get_text()       
                        
                        # обрезаем номера страниц и утверждаем сноски
                        if set(objectTextPart.strip()).issubset(numbers) and pageObjectsKeys[-1] == y1:
                            pageObject['label'] = 'page number'
#                             for textPiece in textObjectsAccumulation[::-1]:
#                                 if textPiece['label'] == 'possible note':
#                                     textPiece['label'] = 'note'
                            continue
                            
                        # это нужно, чтобы собирать заголовки, которые находятся в разных текстовых блоках
                        textObjectsAccumulation.append(pageObject)
                        
                        # это попытка обрезать сноски
#                         if textUnderLine and \
#                            (objectTextPart.split(' ', 1)[0][0] in numbers or \
#                             textObjectsAccumulation[-2]['label'] == 'possible note'):
#                             pageObject['label'] = 'possible note'
#                             continue
                    
                        headerFound = False
                        objectTextParts = list(filter(None,  objectTextPart.split('\n')))
                        
                        if len(textObjectsAccumulation) > 1:
                            prevObjectTextParts = list(filter(None,  textObjectsAccumulation[-2]['object'].get_text().split('\n')))

                            # если заголовок бьется на разные текстовые блоки и является частями больших текстовых блоков
                            if prevObjectTextParts:
                                formattedTextPart = prevObjectTextParts[-1] + objectTextParts[0]
                                formattedTextPart = formattedTextPart.replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                                if formattedTextPart.strip():
                                    if formattedTextPart.strip() == headerPatterns[headerPatternIndex]['headerPattern']:
            #                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex]['headerPattern'])
                                        headerPatterns[headerPatternIndex]['text'] = '' 
                                        # помечаем объект как заголовок
                                        textObjectsAccumulation[::-1][0]['label'] = '(contains) header'
                                        extractText = True
                                        headerFound = True
                                        objectTextPart = '\n'.join(objectTextParts[1:])
                                    if extractText:
            #                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex + 1]['headerPattern'])
                                        # предполагаем, что оглавление задаем порядок следования частей
                                        if headerPatternIndex + 1 != len(headerPatterns) and \
                                           headerPatterns[headerPatternIndex + 1]['headerPattern'] == formattedTextPart.strip():
                                            extractText = False
                                            # contentPageNumber = pageNumber
                                            headerFound = True
                                            objectTextPart = '\n'.join(objectTextParts[1:])
                        
                        if len(textObjectsAccumulation) > 1 and not headerFound:
                            prevObjectTextParts = list(filter(None,  textObjectsAccumulation[-2]['object'].get_text().split('\n')))

                            # иногда заголовок бьется в шахматном порядке
                            # если заголовок бьется на разные текстовые блоки и является частями больших текстовых блоков
                            if prevObjectTextParts:
                                formattedTextPart = objectTextParts[0] + prevObjectTextParts[-1]
                                formattedTextPart = formattedTextPart.replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                                if formattedTextPart.strip():
                                    if formattedTextPart.strip() == headerPatterns[headerPatternIndex]['headerPattern']:
            #                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex]['headerPattern'])
                                        headerPatterns[headerPatternIndex]['text'] = '' 
                                        # помечаем объект как заголовок
                                        textObjectsAccumulation[::-1][0]['label'] = '(contains) header'
                                        extractText = True
                                        headerFound = True
                                        objectTextPart = '\n'.join(objectTextParts[1:])
                                    if extractText:
            #                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex + 1]['headerPattern'])
                                        # предполагаем, что оглавление задаем порядок следования частей
                                        if headerPatternIndex + 1 != len(headerPatterns) and \
                                           headerPatterns[headerPatternIndex + 1]['headerPattern'] == formattedTextPart.strip():
                                            extractText = False
                                            # contentPageNumber = pageNumber
                                            headerFound = True
                                            objectTextPart = '\n'.join(objectTextParts[1:])
                        
#                         marker = False
                        # заголовки по отдельности
                        for objecTextPartIndex in range(0, len(objectTextParts)):
                            formattedTextPart = objectTextParts[objecTextPartIndex].replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                            if not formattedTextPart.strip():
                                continue
#                             if formattedTextPart.strip() == '5.1 Подсчет числа потерь':
#                                 print('I see', objectTextParts, extractText, headerFound, headerPatterns[headerPatternIndex+1]['headerPattern'])
#                                 marker = True
                            if formattedTextPart.strip() == headerPatterns[headerPatternIndex]['headerPattern']:
#                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex]['headerPattern'])
                                headerPatterns[headerPatternIndex]['text'] = '' 
                                # помечаем объект как заголовок
                                textObjectsAccumulation[::-1][0]['label'] = 'header'
                                extractText = True
                                headerFound = True
                                objectTextPart = '\n'.join(objectTextParts[objecTextPartIndex+1:])
                                continue
                            if extractText:
#                                 print(repr(formattedTextPart), headerPatterns[headerPatternIndex + 1]['headerPattern'])
                                # предполагаем, что оглавление задаем порядок следования частей
                                if headerPatternIndex + 1 != len(headerPatterns) and \
                                   headerPatterns[headerPatternIndex + 1]['headerPattern'].startswith(formattedTextPart.strip()):
                                    # contentPageNumber = pageNumber
                                    headerFound = True
                                    objectTextPart = '\n'.join(objectTextParts[objecTextPartIndex+1:])
                                    extractText = False
                                    continue
                                            
                        # проверка на многострочные заголовки
                        textPart = ''
                        for textPieceIndex in range(0, len(textObjectsAccumulation[-3:])):
                            # смысла проверять нет, заголовок уже обнааружен
                            if headerFound:
                                break
                            
                            accumulatedTextParts = list(filter(None, textObjectsAccumulation[::-1][textPieceIndex]['object'].get_text().split('\n')))
                            
                            # предполагаем, что они могут быть частью составного заголовка или могут быть заголовками по отдельности
                            # (!) это не работает, если вторая часть заголовка является частью большого текста
                            for accumulatedTextPart in accumulatedTextParts[::-1]:
                                # часть составного заголовка
                                textPart = accumulatedTextPart.strip() + ' ' + textPart
#                                 print(repr(textPart))
                                formattedTextPart = textPart.replace('-\n', '').replace('\n', ' ').replace('  ', ' ')
                                
                                if not formattedTextPart.strip():
                                    continue
#                                 print(headerPattern['headerPattern'], formattedTextPart)
                                if formattedTextPart.strip() == headerPatterns[headerPatternIndex]['headerPattern']:
                                    headerPatterns[headerPatternIndex]['text'] = '' 
                                    extractText = True
                                    # помечаем объект как заголовок
                                    for headerObjectIndex in range(0, textPieceIndex+1):
                                        textObjectsAccumulation[::-1][headerObjectIndex]['label'] = 'header'
                                    headerFound = True
                                    objectTextPart = '\n'.join(objectTextParts[objecTextPartIndex+1:])
                                    continue
                                if extractText:
                                    # предполагаем, что оглавление задаем порядок следования частей
                                    if headerPatternIndex + 1 != len(headerPatterns) and \
                                       headerPatterns[headerPatternIndex + 1]['headerPattern'].startswith(formattedTextPart.strip()):
                                        extractText = False
                                        # contentPageNumber = pageNumber
                                        headerFound = True
                                        objectTextPart = '\n'.join(objectTextParts[objecTextPartIndex+1:])
                                        continue
                                        
#                         print(headerPatterns[headerPatternIndex]['headerPattern'], headerPatterns[headerPatternIndex + 1]['headerPattern'], headerFound, extractText, objectTextPart)

                        if extractText:
                            headerPatterns[headerPatternIndex]['text'] += formatTextPart(objectTextPart, headerPatterns[headerPatternIndex]['text'])
                            
    for headerPattern in headerPatterns:
        formattedText = re.sub(r'([^ ])\- +', r'\1', headerPattern['text'])
        formattedText = formattedText.replace('\n\n', '---')
        formattedText = formattedText.replace('\n', ' ').replace('  ', ' ')
        formattedText = formattedText.replace('---', '\n')
        while formattedText != formattedText.replace('*formula placeholder*. *formula placeholder*.', '*formula placeholder*.'):
            formattedText = formattedText.replace('*formula placeholder*. *formula placeholder*.', '*formula placeholder*.')
        headerPattern['text'] = formattedText.strip()
        
    return headerPatterns, pageObjectsByY1List

In [ ]:
headerPatternsAndText, pageObjectsByY1List = splitTextByHeaderPatterns(headerPatterns, pageObjectsByY1List, rememberContentPage, formulas)

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headerPatternsAndText)
    pprint(pageObjectsByY1List)

### Извлекаем куски текста в соотвестсвии с иерархией.

In [ ]:
def splitTextByHeadersHierarchy(headersHierarchy, headerPatternsAndText, prevLevelNumeration):
    levelHeaders = []
    for levelNumeration, nestedHierarchy in headersHierarchy.items():
        numerationDelimiter = '' if prevLevelNumeration == '' else '.'
        delimiter = '' if prevLevelNumeration == '' and levelNumeration == '' else ' '
        levelNumeration = prevLevelNumeration + numerationDelimiter + levelNumeration
        if isinstance(nestedHierarchy[0], list):
            for headerIndex in range(0, nestedHierarchy[0]):
                header = levelNumeration + delimiter + nestedHierarchy[0][headerIndex]['header']
                for headerPatternAndText in headerPatternsAndText:
                    if headerPatternAndText['headerPattern'] == header:
                        nestedHierarchy[0][headerIndex]['text'] = headerPatternAndText['text']
        else:
            header = levelNumeration + delimiter + nestedHierarchy[0]['header']
            for headerPatternAndText in headerPatternsAndText:
                if headerPatternAndText['headerPattern'] == header:
                    nestedHierarchy[0]['text'] = headerPatternAndText['text']
        splitTextByHeadersHierarchy(nestedHierarchy[1], headerPatternsAndText, levelNumeration)

In [ ]:
splitTextByHeadersHierarchy(headersHierarchy, headerPatternsAndText, '')
headersAndTextHierarchy = headersHierarchy

if __name__ == '__main__' and '__file__' not in globals():
    pprint(headersAndTextHierarchy)

In [ ]:
if not (__name__ == '__main__' and '__file__' not in globals()):
    headersHierarchy = None
    rememberContentPage = None
    headerPatterns = None